# Exploration
- astrologi V
- produksi V
- trend nama X
- daerah pendatang X

### column
nama, nik, tempatLahir, jenisKelamin, tps, provinsi, kab_kota, 

kecamatan, desa, newcomer, nama_sunda, nama_jawa, nama_batak, 

bintang, inisial_r, nama_kristen, umur, tahun_lahir, tanggal_produksi, 

hari_produksi, unique_location, unique_hash, lat, long, count, unique_location:1,

 lat:1, long:1, lat_clean, long_clean

In [2]:
import pandas as pd
import duckdb
con = duckdb.connect('entity.db')

In [3]:
from datetime import datetime, timedelta
import plotly.express as px

# Astrologi

In [43]:
bintang = con.sql("SELECT nama, provinsi, kab_kota, kecamatan, desa, bintang, umur FROM entity_with_coordinate2 WHERE bintang IS NOT NULL").fetchdf()
bintang = bintang[bintang.umur<=2000]


In [44]:
bintang['tahun_lahir'] = bintang['umur']
bintang['umur'] = bintang['tahun_lahir'] - 1900
bintang.rename(columns={'bintang':'signs'},inplace=True)
bintang.head(3)

,nama,provinsi,kab_kota,kecamatan,desa,signs,umur,tahun_lahir
0,NYANYAH,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,taurus,49,1949
1,JALALUDIN,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,libra,42,1942
2,AEP SAEPUL MALIK,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,leo,32,1932


In [45]:
bintang_size = bintang.value_counts('signs').to_frame().reset_index()
bintang_size.head(3)

,signs,count
0,cancer,3977572
1,gemini,3099980
2,taurus,2879968


In [46]:
fig = px.pie(bintang_size, values='count', names='signs', title='Zodiac Sign In West Java')
fig.update_traces(textposition='inside', textinfo='percent+label',showlegend=False)
fig.show()

In [47]:
fig = px.bar(bintang_size.head(5).sort_values('count'), y='signs', x='count',width=400,text_auto=True, title='')
fig.show()

In [48]:
clean_coordinate = pd.read_csv('clean_coordinate.csv')
clean_coordinate = clean_coordinate[['provinsi','kab_kota','kecamatan','desa','lat','long']]
clean_coordinate.head(3)

,provinsi,kab_kota,kecamatan,desa,lat,long
0,JAWA BARAT,BANDUNG,MAJALAYA,SUKAMAJU,-7.05177,107.75618
1,JAWA BARAT,CIANJUR,CIRANJANG,CIRANJANG,-6.80765,107.27754
2,JAWA BARAT,BANDUNG,MAJALAYA,BIRU,-7.05323,107.72817


In [49]:
kabkota_coordinate = clean_coordinate.groupby(['provinsi','kab_kota'])[['lat','long']].mean().reset_index()
kabkota_coordinate.head(3)

,provinsi,kab_kota,lat,long
0,JAWA BARAT,BANDUNG,-7.061975,107.589536
1,JAWA BARAT,BANDUNG BARAT,-6.833096,107.420848
2,JAWA BARAT,BEKASI,-6.255207,107.244573


In [50]:
bintang_location = bintang.groupby(['provinsi','kab_kota','tahun_lahir','signs']).nama.count().to_frame().reset_index()
bintang_location.head()

,provinsi,kab_kota,tahun_lahir,signs,nama
0,JAWA BARAT,BANDUNG,1924,aquarius,5397
1,JAWA BARAT,BANDUNG,1924,aries,5483
2,JAWA BARAT,BANDUNG,1924,cancer,7161
3,JAWA BARAT,BANDUNG,1924,gemini,5803
4,JAWA BARAT,BANDUNG,1924,leo,6288


In [51]:
bintang_map = pd.merge(bintang_location,kabkota_coordinate,how='left',on=['provinsi','kab_kota'])
bintang_map.head(3)

,provinsi,kab_kota,tahun_lahir,signs,nama,lat,long
0,JAWA BARAT,BANDUNG,1924,aquarius,5397,-7.061975,107.589536
1,JAWA BARAT,BANDUNG,1924,aries,5483,-7.061975,107.589536
2,JAWA BARAT,BANDUNG,1924,cancer,7161,-7.061975,107.589536


In [52]:
bintang_map.sort_values('tahun_lahir').tahun_lahir.unique()

array([1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
       1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945,
       1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956,
       1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967,
       1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000])

In [53]:
len(bintang_map)

20270

In [ ]:
fig.write_html("path/to/file.html")

In [58]:
bintang_map = bintang_map.sort_values('tahun_lahir')
bintang_map['cumulative_value'] = bintang_map.groupby(['provinsi','kab_kota','lat','long','signs'])['nama'].cumsum()
bintang_map.head(3)

,provinsi,kab_kota,tahun_lahir,signs,nama,lat,long,cumulative_value
0,JAWA BARAT,BANDUNG,1924,aquarius,5397,-7.061975,107.589536,5397
4241,JAWA BARAT,CIANJUR,1924,pisces,3421,-6.964360,107.193911,3421
4242,JAWA BARAT,CIANJUR,1924,sagittarius,3317,-6.964360,107.193911,3317


In [61]:
bintang_map[(bintang_map.kab_kota=='BANDUNG') &( bintang_map.signs=='gemini')]

,provinsi,kab_kota,tahun_lahir,signs,nama,lat,long,cumulative_value
3,JAWA BARAT,BANDUNG,1924,gemini,5803,-7.061975,107.589536,5803
14,JAWA BARAT,BANDUNG,1925,gemini,4848,-7.061975,107.589536,10651
25,JAWA BARAT,BANDUNG,1926,gemini,5850,-7.061975,107.589536,16501
36,JAWA BARAT,BANDUNG,1927,gemini,6323,-7.061975,107.589536,22824
47,JAWA BARAT,BANDUNG,1928,gemini,6395,-7.061975,107.589536,29219
...,...,...,...,...,...,...,...,...
795,JAWA BARAT,BANDUNG,1996,gemini,44,-7.061975,107.589536,249197
806,JAWA BARAT,BANDUNG,1997,gemini,21,-7.061975,107.589536,249218
817,JAWA BARAT,BANDUNG,1998,gemini,51,-7.061975,107.589536,249269
828,JAWA BARAT,BANDUNG,1999,gemini,17,-7.061975,107.589536,249286


In [96]:
fig = px.scatter_mapbox(
    bintang_map[(bintang_map.signs=='gemini')],
    # bintang_map[(bintang_map.signs=='gemini')|(bintang_map.signs=='cancer')],
    lat="lat",
    lon="long",
    size="cumulative_value",
    hover_data=["signs",'cumulative_value','kab_kota'],
    animation_frame="tahun_lahir",
    # color="signs",
    title='Pertumbuhan Gemini 1924 - 2000'
).update_layout(
    title_x=0.5,
    title_y=0.95,
    mapbox={"style": "open-street-map", "zoom":7}, margin={"l": 0, "r": 15, "t": 50, "b": 0}
)

fig.show()

In [97]:
fig.write_html("gemini.html")

# Produksi

In [3]:
produksi = con.sql("SELECT nama, provinsi, kab_kota, kecamatan, desa, tanggal_produksi, hari_produksi, umur FROM entity_with_coordinate2 WHERE tanggal_produksi IS NOT NULL").fetchdf()
produksi = produksi[produksi.umur<=2000]
produksi.head(3)

,nama,provinsi,kab_kota,kecamatan,desa,tanggal_produksi,hari_produksi,umur
0,NYANYAH,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1973-08-01,Wednesday,1949
1,JALALUDIN,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1981-01-04,Sunday,1942
2,AEP SAEPUL MALIK,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1990-11-16,Friday,1932


In [7]:
produksi.groupby('tanggal_produksi').nama.count()

tanggal_produksi
1922-03-27     135
1922-03-28      10
1922-03-29       4
1922-03-30       8
1922-03-31       6
              ... 
1999-03-22    2178
1999-03-23    2245
1999-03-24    2084
1999-03-25    1931
1999-03-26    1877
Name: nama, Length: 28058, dtype: int64

In [41]:
produksi['q'] = produksi['tanggal_produksi'].dt.quarter
produksi['month'] = produksi['tanggal_produksi'].dt.month_name()
produksi['year'] = produksi['tanggal_produksi'].dt.year
produksi.head(3)

,nama,provinsi,kab_kota,kecamatan,desa,tanggal_produksi,hari_produksi,umur,q,month,year,year_group
0,NYANYAH,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1973-08-01,Wednesday,1949,3,August,1973,"(1971, 1980)"
1,JALALUDIN,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1981-01-04,Sunday,1942,1,January,1981,"(1981, 1990)"
2,AEP SAEPUL MALIK,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1990-11-16,Friday,1932,4,November,1990,"(1981, 1990)"


In [26]:
print(produksi.year.min(), produksi.year.max())

1922 1999


In [35]:
year_group = []
for i in range(1,9):
    a = 2000-(i*10) + 1
    b = 2000-((i-1)*10) 
    year_group.append((a,b))
year_group


[(1991, 2000),
 (1981, 1990),
 (1971, 1980),
 (1961, 1970),
 (1951, 1960),
 (1941, 1950),
 (1931, 1940),
 (1921, 1930)]

In [38]:
def find_year_group(year):
    for yg in year_group:
        if year >= yg[0] and year <= yg[1]:
            return yg
find_year_group(1951)

(1951, 1960)

In [39]:
produksi['year_group'] = produksi.year.apply(find_year_group)
produksi.head(3)

,nama,provinsi,kab_kota,kecamatan,desa,tanggal_produksi,hari_produksi,umur,q,month,year,year_group
0,NYANYAH,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1973-08-01,Wednesday,1949,3,8,1973,"(1971, 1980)"
1,JALALUDIN,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1981-01-04,Sunday,1942,1,1,1981,"(1981, 1990)"
2,AEP SAEPUL MALIK,JAWA BARAT,GARUT,BANJARWANGI,WANGUNJAYA,1990-11-16,Friday,1932,4,11,1990,"(1981, 1990)"


In [47]:
quarter = produksi.groupby('q').nama.count().to_frame().reset_index()
quarter['quarter'] = quarter.q.apply(lambda x: {1:'Jan-Mar',2:'Apr-Jun',3:'Jul-Sep',4:'Okt-Des'}[x])
quarter.rename(columns={'nama':'count'},inplace=True)


In [49]:
fig = px.bar(quarter, x='quarter', y='count',text_auto=True, title='Human Production Count and Quartile Distribution 1922-1999')
fig.show()

In [52]:
month_year_group = produksi.groupby(['month','year_group']).nama.count().to_frame().reset_index()
month_year_group.head(3)

,month,year_group,nama
0,April,"(1921, 1930)",4616
1,April,"(1931, 1940)",25544
2,April,"(1941, 1950)",77990


In [53]:
month_year_group.rename(columns={'nama':'count'},inplace=True)

In [57]:
month_year_group['month_num'] = month_year_group.month.apply(
    lambda x: {
        'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,
        'August':8,'September':9,'October':10,'November':11,'December':12
    }[x]
)
month_year_group.sort_values('month_num',inplace=True)

In [62]:
fig = px.line(month_year_group, x='month', y='count', color='year_group',
              title='Monthly Human Production Count by 10-Year Ranges')
fig

In [65]:
daily_year_group = produksi.groupby(['hari_produksi','year_group']).nama.count().to_frame().reset_index()
daily_year_group.rename(columns={'nama':'count'},inplace=True)
daily_year_group['hari_num'] = daily_year_group.hari_produksi.apply(
    lambda x: {
        'Sunday':1,'Monday':2,'Tuesday':3,'Wednesday':4,'Thursday':5,'Friday':6,'Saturday':7
    }[x]
)
daily_year_group.sort_values('hari_num',inplace=True)


In [66]:
fig = px.line(daily_year_group, x='hari_produksi', y='count', color='year_group',
              title='Daily Human Production Count by 10-Year Ranges')
fig